In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
import keras
from keras.preprocessing import image
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.models import load_model
from keras.applications.vgg19 import VGG19,preprocess_input,decode_predictions


In [ ]:
os.listdir('/content/drive/MyDrive/Dataset_Plant_Disease/Veg-dataset/Veg-dataset/train_set')

['Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Tomato___Bacterial_spot',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot']

In [ ]:
train_datagen=ImageDataGenerator(zoom_range=0.5,shear_range=0.3,horizontal_flip=True,preprocessing_function=preprocess_input)
val_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train=train_datagen.flow_from_directory('/content/drive/MyDrive/Dataset_Plant_Disease/Veg-dataset/Veg-dataset/train_set',target_size=(256,256),batch_size=32)

val=val_datagen.flow_from_directory('/content/drive/MyDrive/Dataset_Plant_Disease/Veg-dataset/Veg-dataset/test_set',target_size=(256,256),batch_size=32)

Found 11386 images belonging to 9 classes.
Found 3416 images belonging to 9 classes.


In [ ]:
t_img,label=train.next()

In [ ]:
t_img.shape

(32, 256, 256, 3)

In [ ]:
from keras.layers import Dense,Flatten
from keras.models import Model
from keras.applications.vgg19 import VGG19
import keras

In [ ]:
base_model=VGG19(input_shape=(256,256,3),include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable=False

In [ ]:
base_model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
X=Flatten()(base_model.output)
X=Dense(units=9,activation='softmax')(X)

model=Model(base_model.input,X)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

es=EarlyStopping(monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1)

mc=ModelCheckpoint(filepath='veg.h5',monitor='val_accuracy',min_delta=0.01,patience=3,verbose=1,save_best_only=True)


cb=[es,mc]

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
model.fit_generator(train,steps_per_epoch=16,epochs=50,verbose=1,callbacks=cb,validation_data=val,validation_steps=16)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/50
16/16 [==============================] - ETA: 0s - loss: 11.5591 - accuracy: 0.4707 
Epoch 1: val_accuracy improved from -inf to 0.68359, saving model to veg.h5
16/16 [==============================] - 942s 60s/step - loss: 11.5591 - accuracy: 0.4707 - val_loss: 5.3734 - val_accuracy: 0.6836
Epoch 2/50
16/16 [==============================] - ETA: 0s - loss: 5.5333 - accuracy: 0.7070 
Epoch 2: val_accuracy did not improve from 0.68359
16/16 [==============================] - 931s 60s/step - loss: 5.5333 - accuracy: 0.7070 - val_loss: 9.4837 - val_accuracy: 0.6523
Epoch 3/50
16/16 [==============================] - ETA: 0s - loss: 3.6553 - accuracy: 0.7715 
Epoch 3: val_accuracy improved from 0.68359 to 0.79492, saving model to veg.h5
16/16 [==============================] - 928s 59s/step - loss: 3.6553 - accuracy: 0.7715 - val_loss: 5.1642 - val_accuracy: 0.7949
Epoch 4/50
16/16 [==============================] - ETA: 0s - loss: 4.2116 - accuracy: 0.7949 
Epoch 4: val_accura